In [26]:
import numpy as np

from pydrake.all import (MathematicalProgram, Solve, MonomialBasis,
                         Variables, Polynomial)
import pydrake.symbolic as sym


In [57]:
prog = MathematicalProgram()
deg = 6
# Convergence rate = 1-beta
beta = 0.1
# Indeterminates
x = prog.NewIndeterminates(2, 'x_{k}')
u = prog.NewIndeterminates(1, 'u_{k}')
x_next = prog.NewIndeterminates(2, 'x_{k+1}')

# Dynamics contraint
prog.AddEqualityConstraintBetweenPolynomials(
    Polynomial(x_next[0]),
    Polynomial(1.1*x[0] - 0.1*x[0]*x[1] + u[0])
)
prog.AddEqualityConstraintBetweenPolynomials(
    Polynomial(x_next[1]),
    Polynomial(0.9*x[1] + 0.9*x[0])
)

# A and B matrices
Ak = np.array([[1.1-0.1*x[1],   -0.1*x[0]],
               [0.1         ,   0.9]])
Bk = np.array([1, 0])[:, np.newaxis]

# Monomial basis
v = [monomial.ToExpression() for monomial in MonomialBasis(x, deg)]
v_next = [monomial.ToExpression() for monomial in MonomialBasis(x_next, deg)]
print("v: ", v)
# print(v_next)
dim_v = len(v)
w11c = prog.NewContinuousVariables(dim_v, 'w11c')
w12c = prog.NewContinuousVariables(dim_v, 'w12c')
w22c = prog.NewContinuousVariables(dim_v, 'w22c')
# print("w11c: ", w11c)

W11k = w11c.dot(v)
W12k = w12c.dot(v)
W22k = w22c.dot(v)
Wk = np.array([[W11k, W12k], [W12k, W22k]])
# print("W11k: ", W11k)


W11k_next = w11c.dot(v_next)
W12k_next = w12c.dot(v_next)
W22k_next = w22c.dot(v_next)
Wk_next = np.array([[W11k_next, W12k_next], [W12k_next, W22k_next]])

r = prog.NewContinuousVariables(1, 'r')

l1c = prog.NewContinuousVariables(dim_v, 'l1c')
l2c = prog.NewContinuousVariables(dim_v, 'l2c')

L1k = l1c.dot(v)
L2k = l2c.dot(v)
Lk = np.array([L1k, L2k])[:, np.newaxis]

print("Wk: ", Wk.shape)
print("Ak: ", Ak.shape)
print("Bk: ", Bk.shape)
print("Lk: ", Lk.shape)

prog.AddLinearCost(r[0])
prog.AddLinearConstraint(r[0] >= 0)

v = np.array(v).reshape(-1, 1)
print("v: ", v.shape)
# Question: Bk is 2x1 and Lk is 2x1, should Lk be transposed so that you take the outer product and it becomes 2x2?
cross_diag = Ak @ Wk + Bk @ Lk.T
omega = np.block([[Wk_next, cross_diag],
                 [cross_diag.T, (1-beta)*Wk]])
print("omega: ", omega.shape)
# Question: What is the w and what are it's dimensions? needs to be 2x1...
# prog.AddSosConstraint(v.T @ omega @ v - r * np.eye(4))


v:  [<Expression "pow(x_{k}(1), 6)">, <Expression "(x_{k}(0) * pow(x_{k}(1), 5))">, <Expression "(pow(x_{k}(0), 2) * pow(x_{k}(1), 4))">, <Expression "(pow(x_{k}(0), 3) * pow(x_{k}(1), 3))">, <Expression "(pow(x_{k}(0), 4) * pow(x_{k}(1), 2))">, <Expression "(pow(x_{k}(0), 5) * x_{k}(1))">, <Expression "pow(x_{k}(0), 6)">, <Expression "pow(x_{k}(1), 5)">, <Expression "(x_{k}(0) * pow(x_{k}(1), 4))">, <Expression "(pow(x_{k}(0), 2) * pow(x_{k}(1), 3))">, <Expression "(pow(x_{k}(0), 3) * pow(x_{k}(1), 2))">, <Expression "(pow(x_{k}(0), 4) * x_{k}(1))">, <Expression "pow(x_{k}(0), 5)">, <Expression "pow(x_{k}(1), 4)">, <Expression "(x_{k}(0) * pow(x_{k}(1), 3))">, <Expression "(pow(x_{k}(0), 2) * pow(x_{k}(1), 2))">, <Expression "(pow(x_{k}(0), 3) * x_{k}(1))">, <Expression "pow(x_{k}(0), 4)">, <Expression "pow(x_{k}(1), 3)">, <Expression "(x_{k}(0) * pow(x_{k}(1), 2))">, <Expression "(pow(x_{k}(0), 2) * x_{k}(1))">, <Expression "pow(x_{k}(0), 3)">, <Expression "pow(x_{k}(1), 2)">, <Expre